In [ ]:
!wget -c https://zenodo.org/records/3338373/files/musdb18hq.zip

In [ ]:
!unzip musdb18hq.zip -d musdb18hq

In [1]:
import torch
import torchaudio
from glob import glob
from datasets import Dataset, Features, Audio, Value
from IPython.display import Audio
from einops import rearrange

In [96]:
N = (2**21)
tracks = [path.split("/")[2] for path in glob("musdb18hq/train/*/*.wav")][::5]
for i_track, track in enumerate(tracks):
    mix_path = f"musdb18hq/train/{track}/mixture.wav"
    vocal_path = f"musdb18hq/train/{track}/vocals.wav"
    x,fs = torchaudio.load(mix_path, normalize=False)
    v,fs = torchaudio.load(vocal_path, normalize=False)
    C, L = v.shape
    assert(x.shape == v.shape)
    assert fs==44100
    assert C==2
    if L<=N:
        continue
    B = L//N
    x = x[:,:(B*N)]
    v = v[:,:(B*N)]
    x = rearrange(x, 'C (B N) -> B C N', B=B)
    v = rearrange(v, 'C (B N) -> B C N', B=B)
    
    # remove segments that don't have enough vocals
    p = v.to(torch.float).norm(dim=1).mean(dim=1)
    x = x[p>200]
    v = v[p>200]
    break